# Lab 4, Module 3: Real-World Classification - Penguin Species

## From XOR to Real Animals: Multiclass Neural Networks

**Learning Objectives:**
- Apply neural networks to a real dataset with measurements you can picture
- Understand multiclass classification (3 species, not just binary)
- Compare linear baseline vs. hidden layer model
- See how model complexity affects performance
- Connect high-level Keras/TensorFlow to earlier hand-built gradient descent

**Estimated time:** 15-20 minutes

---

## Section 1: Introduction - From Abstract to Real

**Remember from earlier modules:**
- Module 0-2: You worked with XOR - an abstract pattern problem
- You manually adjusted 9 weights to separate data points
- You saw gradient descent automate this process
- You experimented with momentum and learning rates

**Today's shift to reality:**
- Real dataset: Palmer Penguins (not abstract patterns)
- Real task: Identify species from body measurements
- Real tools: TensorFlow/Keras (industry-standard ML library)
- Real expectations: ~95% accuracy is excellent (not 100%)

### The Palmer Penguins Dataset

The Palmer Penguins dataset contains body measurements of 333 penguins from 3 different species observed on islands near Antarctica:
- **Adelie** (class 0)
- **Chinstrap** (class 1)
- **Gentoo** (class 2)

Each penguin has 4 numeric features:
1. Bill length (mm) — how long the beak is
2. Bill depth (mm) — how tall/thick the beak is
3. Flipper length (mm) — how long the wing-flipper is
4. Body mass (g) — how much the penguin weighs

**The Task:** Given these 4 measurements, predict which species the penguin belongs to.

**Connection to XOR:** Like XOR had 2 inputs (x₁, x₂) and 2 classes, Penguins has 4 inputs and 3 classes. The principles are the same, but now we're solving a real problem!

---

## Section 2: Setup and Package Check

First, let's make sure we have all the packages we need. This cell will work on Google Colab or your local machine.

In [ ]:
# Package check and installation
import sys

print("Checking packages...")
print(f"Python version: {sys.version}")

# Check/install required packages
required_packages = [
    ('numpy', 'numpy'),
    ('matplotlib', 'matplotlib'),
    ('seaborn', 'seaborn'),
    ('sklearn', 'scikit-learn'),
    ('tensorflow', 'tensorflow')
]

missing_packages = []
for module_name, package_name in required_packages:
    try:
        __import__(module_name)
        print(f"\u2713 {module_name} is installed")
    except ImportError:
        print(f"\u2717 {module_name} not found")
        missing_packages.append(package_name)

if missing_packages:
    print(f"\nInstalling missing packages: {', '.join(missing_packages)}")
    import subprocess
    for package in missing_packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("\u2713 All packages installed!")
else:
    print("\n\u2713 All required packages are already installed!")

Now import all the libraries we'll use:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print("\u2713 All imports successful!")

---

## Section 3: Load and Explore the Dataset

Let's load the Iris dataset and see what we're working with.

In [ ]:
# Load Palmer Penguins dataset from seaborn
penguins = sns.load_dataset('penguins')

# Drop rows with missing values
penguins = penguins.dropna(subset=['bill_length_mm', 'bill_depth_mm',
                                    'flipper_length_mm', 'body_mass_g'])

# Extract numeric features and labels
feature_names = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
friendly_names = ['Bill Length (mm)', 'Bill Depth (mm)', 'Flipper Length (mm)', 'Body Mass (g)']
X = penguins[feature_names].values

# Encode species as integers: Adelie=0, Chinstrap=1, Gentoo=2
le = LabelEncoder()
y = le.fit_transform(penguins['species'])
species_names = le.classes_  # ['Adelie', 'Chinstrap', 'Gentoo']

print("="*70)
print("PALMER PENGUINS DATASET OVERVIEW")
print("="*70)
print(f"\nDataset shape:")
print(f"  Features (X): {X.shape} - {X.shape[0]} penguins, {X.shape[1]} measurements each")
print(f"  Labels (y):   {y.shape} - {y.shape[0]} species labels")

print(f"\nFeature names:")
for i, (col, friendly) in enumerate(zip(feature_names, friendly_names)):
    print(f"  {i+1}. {friendly}")

print(f"\nTarget classes:")
for i, name in enumerate(species_names):
    count = np.sum(y == i)
    print(f"  Class {i}: {name:15s} - {count} samples")

print(f"\nSample data (first 3 penguins):")
print(f"  {'Bill L':>7s}  {'Bill D':>7s}  {'Flip L':>7s}  {'Mass':>7s}  Species")
for i in range(3):
    print(f"  {X[i][0]:7.1f}  {X[i][1]:7.1f}  {X[i][2]:7.1f}  {X[i][3]:7.0f}  {species_names[y[i]]}")
print("="*70)

### Visualize: Can We See the Species?

Let's plot two of the features to see if the species are visually separable.

In [ ]:
# Create scatter plot using flipper length vs body mass
fig, ax = plt.subplots(figsize=(10, 7), dpi=100)

# Plot each class with different color
colors = ['blue', 'red', 'green']
for i, (name, color) in enumerate(zip(species_names, colors)):
    mask = (y == i)
    ax.scatter(X[mask, 2], X[mask, 3],
              c=color, label=name, s=100, alpha=0.7,
              edgecolors='k', linewidths=1.5)

ax.set_xlabel('Flipper Length (mm)', fontsize=13, fontweight='bold')
ax.set_ylabel('Body Mass (g)', fontsize=13, fontweight='bold')
ax.set_title('Palmer Penguins: Flipper Length vs Body Mass by Species',
            fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("   - Gentoo (green) is clearly separated — large flippers and heavy")
print("   - Adelie (blue) and Chinstrap (red) overlap somewhat")
print("   - This is like XOR, but with 3 classes instead of 2!")
print("   - A neural network should be able to learn these boundaries")

---

## Section 4: Train/Test Split and Feature Scaling

**Why split the data?**
- **Training set (80%):** Used to teach the network
- **Test set (20%):** Used to evaluate how well it learned (never seen during training)

**Why scale features?**
- Features have very different units and ranges (e.g., flipper: 172-231 mm, mass: 2700-6300 g)
- Neural networks learn better when all features are on similar scales
- StandardScaler transforms each feature to mean=0, std=1

In [ ]:
# Split into train (80%) and test (20%) sets
# stratify=y ensures each class is proportionally represented in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data split:")
print(f"  Training set:   {X_train.shape[0]} samples")
print(f"  Test set:       {X_test.shape[0]} samples")

# Scale features to mean=0, std=1
# IMPORTANT: Fit scaler on training data only, then apply to both train and test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nFeature scaling:")
print(f"  Before scaling - mean: {X_train[:, 0].mean():.2f}, std: {X_train[:, 0].std():.2f}")
print(f"  After scaling  - mean: {X_train_scaled[:, 0].mean():.2f}, std: {X_train_scaled[:, 0].std():.2f}")
print("\n✓ Data prepared for training!")

---

## Section 5: Baseline Linear Model (No Hidden Layer)

**Connection to Module 2:**
- Remember the XOR network with NO hidden neurons? It couldn't learn XOR.
- A linear model draws straight boundaries in feature space
- Let's see how well this works for Penguins!

**Model architecture:**
- Input: 4 features
- Output: 3 units (one per class) with softmax activation
- NO hidden layers - just a direct linear transformation

This is like logistic regression, but for 3 classes instead of 2.

In [ ]:
# Build linear model (no hidden layer)
linear_model = Sequential([
    Dense(3, activation='softmax', input_dim=4, name='output')
], name='Linear_Model')

# Compile model
# - Adam optimizer: like momentum from Module 2, but smarter!
# - sparse_categorical_crossentropy: loss function for multiclass classification
# - accuracy: % of correct predictions
linear_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Show model architecture
print("="*70)
print("LINEAR MODEL ARCHITECTURE")
print("="*70)
linear_model.summary()
print("\nThis model has NO hidden layers - just 4 inputs \u2192 3 outputs")
print("   Total trainable parameters: (4 features \u00d7 3 classes) + 3 biases = 15")

### Train the Linear Model

**What's happening during training:**
- The `.fit()` method does gradient descent automatically!
- It computes gradients (backpropagation) just like you saw in Module 2
- Adam optimizer updates weights using momentum and adaptive learning rates
- All this happens behind the scenes - no manual weight adjustment needed!

In [ ]:
# Train the model
print("Training linear model...\n")

history_linear = linear_model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=0  # Suppress detailed output
)

print("✓ Training complete!")

### Evaluate on Test Set

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = linear_model.evaluate(X_test_scaled, y_test, verbose=0)

print("="*70)
print("LINEAR MODEL RESULTS")
print("="*70)
print(f"Test Accuracy:  {test_accuracy:.1%}")
print(f"Test Loss:      {test_loss:.4f}")
print("="*70)

if test_accuracy >= 0.95:
    print("\nExcellent! Linear model works well on Penguins.")
    print("   Why? The classes are mostly linearly separable.")
elif test_accuracy >= 0.90:
    print("\nGood performance, but not perfect.")
    print("  The overlapping classes are hard to separate with straight boundaries.")
else:
    print("\nLinear model struggles with this data.")
    print("  We'll need hidden layers to improve!")

### Visualize Training Progress

In [ ]:
# Plot training and validation accuracy
try:
    history_linear
except NameError:
    raise RuntimeError(
        "history_linear is not defined. Please run the 'Train the Linear Model' "
        "cell above (Section 5) before running this cell."
    )

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# Accuracy plot
ax1.plot(history_linear.history['accuracy'], label='Training', linewidth=2)
ax1.plot(history_linear.history['val_accuracy'], label='Validation', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Linear Model: Accuracy over Training', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history_linear.history['loss'], label='Training', linewidth=2)
ax2.plot(history_linear.history['val_loss'], label='Validation', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title('Linear Model: Loss over Training', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nWhat to look for:")
print("   - Accuracy increases quickly in early epochs")
print("   - Training and validation curves are close (no overfitting)")
print("   - Loss decreases and stabilizes")

---

## Section 6: Model with Hidden Layer

**Connection to Module 1:**
- Remember the 2-2-1 network for XOR? It had 2 hidden neurons.
- Hidden layers let the network learn **curved boundaries** (not just straight lines)
- More hidden neurons = more flexibility (but also more to learn)

**Experiment:** Try different numbers of hidden units and see how it affects performance!

In [ ]:
# Adjustable parameter - TRY CHANGING THIS!
hidden_units = 8  # Try: 2, 4, 8, 16, 32, 64

# Build model with one hidden layer
hidden_model = Sequential([
    Dense(hidden_units, activation='relu', input_dim=4, name='hidden'),
    Dense(3, activation='softmax', name='output')
], name='Hidden_Layer_Model')

# Compile with same settings as linear model
hidden_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("="*70)
print(f"MODEL WITH {hidden_units} HIDDEN UNITS")
print("="*70)
hidden_model.summary()
print(f"\nThis model has a hidden layer with {hidden_units} neurons")
print("   Each hidden neuron creates a new 'dimension' (like x\u2083 = x\u2081 \u00d7 x\u2082 for XOR)")
print("   ReLU activation allows curved boundaries (like tanh in Module 1)")

### Train the Hidden Layer Model

In [ ]:
# Train the model
print(f"Training model with {hidden_units} hidden units...\n")

history_hidden = hidden_model.fit(
    X_train_scaled, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=0
)

print("✓ Training complete!")

### Evaluate and Compare

In [ ]:
# Evaluate on test set
test_loss_hidden, test_accuracy_hidden = hidden_model.evaluate(X_test_scaled, y_test, verbose=0)

print("="*70)
print("COMPARISON: LINEAR vs HIDDEN LAYER MODEL")
print("="*70)
print(f"Linear Model (no hidden layer):    {test_accuracy:.1%}")
print(f"Hidden Layer Model ({hidden_units:2d} units):     {test_accuracy_hidden:.1%}")
print("="*70)

improvement = test_accuracy_hidden - test_accuracy
if improvement > 0.02:
    print(f"\nHidden layer helps! Improvement: +{improvement:.1%}")
elif improvement > 0:
    print(f"\nSlight improvement: +{improvement:.1%}")
else:
    print(f"\nNo improvement. Linear model was already good enough!")
    print("   This suggests Penguins data is mostly linearly separable.")

print(f"\nTry changing 'hidden_units' above and re-running to experiment!")
print("   Does more neurons always help? Where do you see diminishing returns?")

### Visualize Hidden Layer Model Training

In [ ]:
# Plot training curves for hidden layer model
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# Accuracy plot
ax1.plot(history_hidden.history['accuracy'], label='Training', linewidth=2)
ax1.plot(history_hidden.history['val_accuracy'], label='Validation', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title(f'Hidden Layer Model ({hidden_units} units): Accuracy', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history_hidden.history['loss'], label='Training', linewidth=2)
ax2.plot(history_hidden.history['val_loss'], label='Validation', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title(f'Hidden Layer Model ({hidden_units} units): Loss', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Section 8: Understanding Variability - ML is an Experiment!

### Why Run Multiple Experiments?

**You may have noticed:** Running the same model twice can give different results!

**Why does this happen?**
- Neural networks start with **random initial weights**
- Different starting points can lead to different final solutions
- This is called the **stochastic nature** of machine learning

**Connection to Module 2:**
- Remember when you tried different random starting points for XOR?
- Some converged faster, some to slightly different loss values
- Same thing happens here - but Keras handles it automatically!

### The Scientific Approach

**In real ML research and practice:**
- You NEVER report results from a single run
- You run the experiment multiple times (5, 10, or even 100 times)
- You report the **mean (average)** and **standard deviation (spread)**
- This tells you: "How reliable is this model?"

**Let's do this properly!** Run multiple experiments and analyze the results statistically.

---

### Experiment: Linear Model with Multiple Runs

Let's train the linear model **5 times** and see how much the results vary.

In [ ]:
# Run linear model multiple times to understand variability
num_runs = 5
num_epochs = 70

print("="*70)
print(f"RUNNING LINEAR MODEL {num_runs} TIMES")
print("="*70)
print("\nWhy? To see how much results vary due to random initialization!\n")

# Store results from each run
linear_test_accuracies = []
linear_test_losses = []
linear_histories = []

for run in range(num_runs):
    print(f"Run {run+1}/{num_runs}...", end=" ")
    
    # IMPORTANT: Create a NEW model each time!
    # This ensures fresh random initialization
    linear_model_run = Sequential([
        Dense(3, activation='softmax', input_dim=4, name='output')
    ], name=f'Linear_Model_Run_{run+1}')
    
    linear_model_run.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model
    history = linear_model_run.fit(
        X_train_scaled, y_train,
        epochs=num_epochs,
        batch_size=16,
        validation_split=0.2,
        verbose=0
    )
    
    # Evaluate on test set
    test_loss, test_accuracy = linear_model_run.evaluate(X_test_scaled, y_test, verbose=0)
    
    # Store results
    linear_test_accuracies.append(test_accuracy)
    linear_test_losses.append(test_loss)
    linear_histories.append(history)
    
    print(f"Accuracy: {test_accuracy:.1%}, Loss: {test_loss:.4f}")

print("\n" + "="*70)
print("STATISTICAL SUMMARY - LINEAR MODEL")
print("="*70)
print(f"Test Accuracy:  {np.mean(linear_test_accuracies):.1%} ± {np.std(linear_test_accuracies):.1%}")
print(f"Test Loss:      {np.mean(linear_test_losses):.4f} ± {np.std(linear_test_losses):.4f}")
print(f"\nMin Accuracy:   {np.min(linear_test_accuracies):.1%}")
print(f"Max Accuracy:   {np.max(linear_test_accuracies):.1%}")
print(f"Range:          {np.max(linear_test_accuracies) - np.min(linear_test_accuracies):.1%}")
print("="*70)

print("\n💡 What does this tell us?")
if np.std(linear_test_accuracies) < 0.05:
    print("   - Low standard deviation → Results are CONSISTENT")
    print("   - The model is stable across different initializations")
else:
    print("   - Higher standard deviation → Results VARY")
    print("   - Random initialization matters more for this model")
print(f"   - We can report: {np.mean(linear_test_accuracies):.1%} ± {np.std(linear_test_accuracies):.1%}")

### Visualize All Runs Together

In [ ]:
# Plot training curves from all runs on the same plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# Accuracy plot - all runs
for i, history in enumerate(linear_histories):
    ax1.plot(history.history['val_accuracy'], alpha=0.6, linewidth=2, label=f'Run {i+1}')

ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Validation Accuracy', fontsize=12, fontweight='bold')
ax1.set_title(f'Linear Model: {num_runs} Runs - Validation Accuracy', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Loss plot - all runs
for i, history in enumerate(linear_histories):
    ax2.plot(history.history['val_loss'], alpha=0.6, linewidth=2, label=f'Run {i+1}')

ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Validation Loss', fontsize=12, fontweight='bold')
ax2.set_title(f'Linear Model: {num_runs} Runs - Validation Loss', fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 What to observe:")
print("   - Do all runs converge to similar accuracy?")
print("   - Do some runs converge faster than others?")
print("   - Is there a spread in final performance?")
print("   - This variability is NORMAL and expected in ML!")

### Experiment: Hidden Layer Model with Multiple Runs

Now let's do the same for the hidden layer model. Do models with more parameters vary more or less?

In [ ]:
# Adjustable parameter - change this to experiment!
hidden_units_experiment = 16  # Try: 8, 16, 32

print("="*70)
print(f"RUNNING HIDDEN LAYER MODEL ({hidden_units_experiment} units) {num_runs} TIMES")
print("="*70)
print()

# Store results from each run
hidden_test_accuracies = []
hidden_test_losses = []
hidden_histories = []

for run in range(num_runs):
    print(f"Run {run+1}/{num_runs}...", end=" ")
    
    # Create a NEW model each time for fresh initialization
    hidden_model_run = Sequential([
        Dense(hidden_units_experiment, activation='relu', input_dim=4, name='hidden'),
        Dense(3, activation='softmax', name='output')
    ], name=f'Hidden_Model_Run_{run+1}')
    
    hidden_model_run.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Train the model
    history = hidden_model_run.fit(
        X_train_scaled, y_train,
        epochs=num_epochs,
        batch_size=16,
        validation_split=0.2,
        verbose=0
    )
    
    # Evaluate on test set
    test_loss, test_accuracy = hidden_model_run.evaluate(X_test_scaled, y_test, verbose=0)
    
    # Store results
    hidden_test_accuracies.append(test_accuracy)
    hidden_test_losses.append(test_loss)
    hidden_histories.append(history)
    
    print(f"Accuracy: {test_accuracy:.1%}, Loss: {test_loss:.4f}")

print("\n" + "="*70)
print(f"STATISTICAL SUMMARY - HIDDEN LAYER MODEL ({hidden_units_experiment} units)")
print("="*70)
print(f"Test Accuracy:  {np.mean(hidden_test_accuracies):.1%} ± {np.std(hidden_test_accuracies):.1%}")
print(f"Test Loss:      {np.mean(hidden_test_losses):.4f} ± {np.std(hidden_test_losses):.4f}")
print(f"\nMin Accuracy:   {np.min(hidden_test_accuracies):.1%}")
print(f"Max Accuracy:   {np.max(hidden_test_accuracies):.1%}")
print(f"Range:          {np.max(hidden_test_accuracies) - np.min(hidden_test_accuracies):.1%}")
print("="*70)

### Visualize Hidden Layer Model Runs

In [ ]:
# Plot training curves from all runs
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# Accuracy plot - all runs
for i, history in enumerate(hidden_histories):
    ax1.plot(history.history['val_accuracy'], alpha=0.6, linewidth=2, label=f'Run {i+1}')

ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Validation Accuracy', fontsize=12, fontweight='bold')
ax1.set_title(f'Hidden Layer Model ({hidden_units_experiment}U): {num_runs} Runs - Validation Accuracy', 
             fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Loss plot - all runs
for i, history in enumerate(hidden_histories):
    ax2.plot(history.history['val_loss'], alpha=0.6, linewidth=2, label=f'Run {i+1}')

ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Validation Loss', fontsize=12, fontweight='bold')
ax2.set_title(f'Hidden Layer Model ({hidden_units_experiment}U): {num_runs} Runs - Validation Loss', 
             fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Final Comparison: Linear vs Hidden Layer

In [ ]:
# Statistical comparison
print("="*70)
print("FINAL COMPARISON: LINEAR vs HIDDEN LAYER (STATISTICAL)")
print("="*70)
print(f"\nLinear Model (no hidden layer):")
print(f"  Mean Accuracy:  {np.mean(linear_test_accuracies):.1%} ± {np.std(linear_test_accuracies):.1%}")
print(f"  Mean Loss:      {np.mean(linear_test_losses):.4f} ± {np.std(linear_test_losses):.4f}")

print(f"\nHidden Layer Model ({hidden_units_experiment} units):")
print(f"  Mean Accuracy:  {np.mean(hidden_test_accuracies):.1%} ± {np.std(hidden_test_accuracies):.1%}")
print(f"  Mean Loss:      {np.mean(hidden_test_losses):.4f} ± {np.std(hidden_test_losses):.4f}")

mean_improvement = np.mean(hidden_test_accuracies) - np.mean(linear_test_accuracies)
print(f"\nMean Improvement: {mean_improvement:+.1%}")
print("="*70)

# Create box plot to visualize distributions
fig, ax = plt.subplots(figsize=(10, 6), dpi=100)

box_data = [linear_test_accuracies, hidden_test_accuracies]
box_labels = ['Linear Model\n(0 hidden layers)', f'Hidden Layer Model\n({hidden_units_experiment} units)']

bp = ax.boxplot(box_data, labels=box_labels, patch_artist=True,
                showmeans=True, meanline=True)

# Color the boxes
colors = ['lightblue', 'lightgreen']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title(f'Test Accuracy Distribution Across {num_runs} Runs', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add individual points
for i, data in enumerate(box_data, 1):
    x = np.random.normal(i, 0.04, size=len(data))
    ax.scatter(x, data, alpha=0.6, s=60, c='red', edgecolors='black', linewidths=1)

plt.tight_layout()
plt.show()

print("\n💡 Understanding the box plot:")
print("   - Box shows the middle 50% of results (interquartile range)")
print("   - Line in box is the median")
print("   - Green dashed line is the mean")
print("   - Red dots are individual run results")
print("   - Whiskers show the full range (min to max)")
print("\n   → This is how you should report ML results in research!")

### Key Insights from Multiple Runs

**What you should have learned:**

1. **ML is stochastic** - Same code, different results each time!
   - Random weight initialization matters
   - Results vary from run to run

2. **Reporting means ± std is standard practice**
   - Never trust a single run
   - Always run experiments multiple times
   - Report: "Accuracy: 93.2% ± 2.1%" not "Accuracy: 95.0%"

3. **Variability can tell you about model stability**
   - Low std → Consistent, reliable model
   - High std → Sensitive to initialization, may need more tuning

4. **Statistical comparison is more robust**
   - Comparing single runs: "95% vs 93%" (unreliable!)
   - Comparing distributions: "94.5±1.2% vs 92.8±2.3%" (reliable!)

**Connection to Module 2:**
- Remember testing multiple random starting points for XOR?
- You saw some converged faster, some to different local minima
- **Same concept here**, but now you're quantifying it statistically!

---

---

## Section 9: Connection to Earlier Labs

Let's explicitly connect what we just did to the gradient descent work from Modules 0-2!

### What Just Happened Behind the Scenes?

**From Module 2 - Training with Gradient Descent:**

Remember when you saw:
```python
# Manual gradient descent (Module 2)
for epoch in range(max_epochs):
    grads = compute_gradients(network, X, y)
    weights = weights - learning_rate * grads
```

**Keras `.fit()` does this automatically:**
- Computes gradients using backpropagation (same math you saw in Module 2!)
- Updates weights using Adam optimizer (like momentum, but smarter)
- Loops through epochs and batches
- Tracks training/validation metrics

### Adam Optimizer = Gradient Descent with Momentum + More

**From Module 2, you saw:**
```python
velocity = beta * velocity + (1-beta) * gradient  # Momentum
weights = weights - learning_rate * velocity
```

**Adam does:**
- Momentum (like you saw)
- Adaptive learning rates (different rate for each weight)
- Bias correction
- All automatically tuned!

### Multi-Start Initialization

**From Module 2:**
- You manually tried multiple random starting points
- Picked the one with lowest initial loss

**Keras does this automatically:**
- Uses smart initialization (Xavier/He initialization)
- Weights start in a good range for the activation function
- No need to manually restart!

### The Big Picture

```
Module 0: Learned that lifting to higher dimensions helps (x₃ = x₁ × x₂)
         → Hidden layers do this automatically!

Module 1: Manually adjusted 9 weights to solve XOR
         → .fit() does this automatically with gradient descent!

Module 2: Saw gradient descent with momentum converge automatically
         → Adam optimizer is even better!

Module 3: Applied these same principles to REAL data (penguin species)
         → Same math, real problems, industrial-strength tools!
```

**The fundamental concepts haven't changed - we're just using powerful tools that automate the process!**

---

## Section 10: Key Takeaways

### 1. Real-World ML Works!
- Neural networks can classify real data (penguin species) with high accuracy
- 95%+ accuracy on Penguins is excellent (not 100% - that would be suspicious!)
- The principles from XOR apply to real problems

### 2. Linear Models Can Be Sufficient
- Penguins data is mostly linearly separable → linear model works well
- Hidden layers help, but the improvement may be modest
- Not every problem needs a deep network!

### 3. Hidden Layers Add Flexibility
- Hidden neurons create new "dimensions" (like x₃ = x₁ × x₂ for XOR)
- More units = more flexibility, but diminishing returns
- There's a sweet spot - too few underfits, too many may overfit

### 4. Keras/TensorFlow Automates Gradient Descent
- `.fit()` does backpropagation and weight updates automatically
- Adam optimizer is like momentum (Module 2) but better
- All the math you learned still applies - it's just hidden!

### 5. Real Data Has Realistic Accuracy
- Perfect accuracy (100%) is rare and may indicate overfitting
- 90-98% is typical for well-separated classes
- Some errors are expected due to noise and overlap


### 6. ML is Stochastic - Always Run Multiple Experiments!
- Random initialization causes result variability
- Professional ML practice: run multiple times, report mean ± std
- Low variability = stable, reliable model

---

## Reflection Questions

Answer these on your answer sheet:

**Q1.** Did the linear model (no hidden layer) achieve high accuracy on Penguins? Why or why not?

**Q2.** How much did adding a hidden layer improve accuracy? Was the improvement large or small?

**Q3.** Experiment with different values of `hidden_units` (2, 4, 8, 16, 32, 64). At what point do you see diminishing returns?

**Q4.** Compare this to Module 2 XOR training:
   - What's similar? (Hint: gradient descent, weight updates)
   - What's different? (Hint: manual vs. automatic, dataset)

**Q5.** Why is 95% accuracy considered "excellent" while 100% might be suspicious?

**Q6.** Looking at the flipper-vs-mass scatter plot, why do you think Adelie and Chinstrap are harder to separate than Gentoo?

---

## Next Steps

1. **Experiment** with `hidden_units` above - try 2, 8, 16, 32, 64
2. **Record** your results: how does test accuracy change?
3. **Answer** the reflection questions on your answer sheet
4. **Return to the LMS** and continue to Module 4

In **Module 4**, you'll work with a medical diagnosis dataset (breast cancer detection) with 30 features instead of 4. You'll experiment with model architecture (number of layers and units) and learn about confusion matrices for medical applications.

---

**Great job! You've successfully applied neural networks to real-world data!**